# Praca domowa nr 4

Budowanie modelu na podstawie artykułu: https://academic.oup.com/jamiaopen/article/1/1/87/5032901

Przed operacjami w tym notebooku, został użyty skrypt do preprecessingu załączony do wymienionego wyżej artykułu, który jest dostępny tutaj: https://github.com/illidanlab/urgent-care-comparative

Zadanie: problem klasyfikacji, predykcja śmiertelności na podstawie przedstawienia danych w postaci *X48* (wg. artykułu powyżej).

### Biblioteki + dodatkowe funkcje

In [1]:
import numpy as np
import pandas as pd

import pickle

from sklearn.model_selection import *
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import roc_curve, auc as auc_score, confusion_matrix, f1_score
from sklearn.utils import shuffle

In [2]:
# funkcja z repozytorium reprodukowanego artykułu
def single_score(y_te, yhat):
    fpr, tpr, thresholds = roc_curve(y_te, yhat)
    roc_auc = auc_score(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    yhat[yhat>=optimal_threshold]=1; yhat[yhat<optimal_threshold]=0
    yhat=[int(i) for i in yhat]
    #matrix = confusion_matrix(y_te, yhat)
    tn, fp, fn, tp = confusion_matrix(y_te, yhat).ravel()
    sen=1.0* (tp/(tp+fn))
    spec=1.0* (tn/(tn+fp))
    f1=f1_score(y_te,yhat)
    return roc_auc, f1, sen, spec

In [3]:
# funkcja z repozytorium reprodukowanego artykułu
def balanced_subsample(x,y,subsample_size=1.0):
    class_xs = []
    min_elems = None
    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]
    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)
    xs = []
    ys = []
    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

### Załadowanie danych po preprocessingu

In [4]:
X = np.load("X48.npy")
X

array([[4.22535211e-02, 0.00000000e+00, 5.43478261e-02, ...,
        4.03017024e-01, 1.33952979e-01, 4.75067826e-01],
       [2.58215962e-01, 1.18421053e-01, 3.26086957e-01, ...,
        4.10958588e-14, 0.00000000e+00, 0.00000000e+00],
       [2.11267606e-01, 4.05553814e-01, 8.69565217e-02, ...,
        2.52207581e-01, 1.88907108e-01, 2.31845699e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.10958588e-14, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.10526316e-01, 0.00000000e+00, ...,
        2.64952543e-01, 2.88610525e-01, 4.89848547e-02],
       [3.20522201e-01, 1.57894737e-01, 0.00000000e+00, ...,
        4.83805376e-02, 3.83338758e-02, 1.06946021e-01]])

In [5]:
with open('y', 'rb') as f:
    labels = pickle.load(f)
    
task = [yy[0] for yy in labels]
y = np.array(task)
y

array([0, 0, 0, ..., 0, 0, 0])

### Modelowanie z kroswalidacją

#### Random Forest

In [6]:
skf = StratifiedKFold(n_splits = 5)
count = 0
data = [None] * 5
for train_index, test_index in skf.split(X, y):
    print("Iteration: ", count)
    print("TRAIN:", train_index, "TEST:", test_index)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # model = RF(n_estimators = 450, verbose = 1)
    model = RF(n_estimators = 450)
    
    xs, ys = balanced_subsample(X_train, y_train, 1)
    
    model = model.fit(xs, ys)
    
    yhat = model.predict(X_train)
    tr_auc, _, _, _ = single_score(y_train, yhat)
    
    yhat2 = model.predict(X_test)
    te_auc, f1_scor, sen, spec = single_score(y_test, yhat2)
    
    data[count] = {'train_auc' : tr_auc, 'test_auc' : te_auc, 'test_f1_score' : f1_scor, 'test_sen(precision)' : sen, 'test_spec(recall)' : spec}
    count += 1

Iteration:  0
TRAIN: [ 4964  4971  4994 ... 27613 27614 27615] TEST: [   0    1    2 ... 5602 5603 5604]
Iteration:  1
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [ 4964  4971  4994 ... 11217 11218 11219]
Iteration:  2
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [ 9861  9876  9893 ... 16776 16777 16778]
Iteration:  3
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [14885 14892 14917 ... 22221 22222 22223]
Iteration:  4
TRAIN: [    0     1     2 ... 22221 22222 22223] TEST: [20978 20997 21005 ... 27613 27614 27615]


In [7]:
pd.DataFrame(data).describe()

,train_auc,test_auc,test_f1_score,test_sen(precision),test_spec(recall)
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.885757,0.768931,0.426632,0.804203,0.733658
std,0.001229,0.010809,0.008064,0.036333,0.019665
min,0.884598,0.755724,0.418002,0.755690,0.709147
25%,0.884951,0.762562,0.420418,0.792109,0.721069
50%,0.885163,0.770355,0.424733,0.793627,0.731497
75%,0.886576,0.771466,0.433735,0.831563,0.750822
max,0.887496,0.784547,0.436272,0.848024,0.755757


#### GradientBoostingClassifier

In [8]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

skf = StratifiedKFold(n_splits = 5)
count = 0
data1 = [None] * 5
for train_index, test_index in skf.split(X, y):
    print("Iteration: ", count)
    print("TRAIN:", train_index, "TEST:", test_index)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # model = GBC(n_estimators = 400, learning_rate = 0.09, verbose = 1)
    model = GBC(n_estimators = 400, learning_rate = 0.09)
    
    xs, ys = balanced_subsample(X_train, y_train, 1)
    
    model = model.fit(xs, ys)
    
    yhat = model.predict(X_train)
    tr_auc, _, _, _ = single_score(y_train, yhat)
    
    yhat2 = model.predict(X_test)
    te_auc, f1_scor, sen, spec = single_score(y_test, yhat2)
    
    data1[count] = {'train_auc' : tr_auc, 'test_auc' : te_auc, 'test_f1_score' : f1_scor, 'test_sen(precision)' : sen, 'test_spec(recall)' : spec}
    count += 1

Iteration:  0
TRAIN: [ 4964  4971  4994 ... 27613 27614 27615] TEST: [   0    1    2 ... 5602 5603 5604]
Iteration:  1
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [ 4964  4971  4994 ... 11217 11218 11219]
Iteration:  2
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [ 9861  9876  9893 ... 16776 16777 16778]
Iteration:  3
TRAIN: [    0     1     2 ... 27613 27614 27615] TEST: [14885 14892 14917 ... 22221 22222 22223]
Iteration:  4
TRAIN: [    0     1     2 ... 22221 22222 22223] TEST: [20978 20997 21005 ... 27613 27614 27615]


In [9]:
pd.DataFrame(data1).describe()

,train_auc,test_auc,test_f1_score,test_sen(precision),test_spec(recall)
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.848286,0.767540,0.438640,0.772024,0.763056
std,0.004128,0.010340,0.009578,0.035060,0.020267
min,0.841840,0.758347,0.426819,0.729894,0.739979
25%,0.846722,0.759013,0.430610,0.755690,0.746557
50%,0.849729,0.766044,0.441893,0.761760,0.762336
75%,0.851059,0.770683,0.444262,0.792109,0.779605
max,0.852081,0.783613,0.449619,0.820669,0.786801


### Porównanie z wynikami z artykułu

![Wyniki z artykułu](results_from_article.jpg)

* Otrzymane przez nas wyniki (patrząc na średnią) są porównywalne z tymi, które zostały przedstawione w artykule. W przypadku GradientBoostingClassifier jest to różnica kilku setnych, jednak w przypadku RandomForest jest to nawet około jednej dziesiątej.

* Nie są to jednak te same wyniki. Jest to spowodowane tym, że autorzy artykułu nie podali optymalnych hiperparametrów dla tych modeli. 

* W kodzie na githubie modele są inicjowane z pewnymi hiperparametrami, jednak nie są to te, które dają optymalne wyniki zamieszczone w artykule. 

* Okazuje się również, że w dodatku do artukułu podano najbardziej optymalne hiperparametry dla niektórych modeli (np. SVM). Jednakże, nawet w tym przypadku wartości nie pokrywają się z tymi, które są w kodzie dla tego modelu.

### Zapisanie wyników

In [10]:
with open('raw_stats_rf.npy', 'wb') as f:
    pickle.dump(data, f)

with open('raw_stats_gbc.npy', 'wb') as f:
    pickle.dump(data1, f)

### Sprawdzenie czasów uczenia modeli 
Szybciej uczące się modele mogą być lepsze, gdy będziemy tworzyć zbiory Rashomon

In [11]:
def measure_time(model):
    import time
    start_time = time.time()
    
    model.fit(X, y)

    end_time = time.time()
    return end_time-start_time

In [16]:
import xgboost as xgb

print(f"Random Forest: {measure_time(RF(n_estimators = 450, n_jobs = -1))}")
print(f"GradientBoostingClassifier: {measure_time(GBC(n_estimators = 400, learning_rate = 0.09))}")
print(f"XGBClassifier: {measure_time(xgb.XGBClassifier(objective='binary:logistic', n_estimators = 400, learning_rate = 0.09, use_label_encoder=False, n_jobs = -1))}")

Random Forest: 14.911872625350952
GradientBoostingClassifier: 140.62497067451477
[13:50:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier: 21.846973180770874


RandomForest oraz XGBClassifier może działać wielowątkowo, dlatego są one szybsze.